In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

In [2]:
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3060 Laptop GPU, compute capability 8.6
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale
Compute dtype: float16
Variable dtype: float32


In [4]:
train_dir = "D:/Ocular toxoplasmosis binzarized/Oculur Toxoplasmosis/train_augmented"
test_dir = "D:/Ocular toxoplasmosis binzarized/Oculur Toxoplasmosis/val"

In [5]:
epochs = 10
input_shape = (112, 112)

datagen_train = ImageDataGenerator(rescale=1./255,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  horizontal_flip=True,
                                  vertical_flip=False)


datagen_test = ImageDataGenerator(rescale=1./255)


batch_size = 16
generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True)

generator_test = datagen_test.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)
steps_per_epoch = generator_train.n / batch_size
steps_test = generator_test.n / batch_size

Found 3659 images belonging to 2 classes.
Found 83 images belonging to 2 classes.


In [6]:
def model_creator(model):
    transfer_layer = model.get_layer(model.get_layer(index = -1).name)
    conv_model = Model(inputs=model.input, outputs=transfer_layer.output)
    new_model = Sequential()
    new_model.add(conv_model)
    new_model.add(Flatten())
    new_model.add(Dense(2, activation='softmax'))
    optimizer = Adam(lr=1e-5)
    loss = 'categorical_crossentropy'
    metrics = ['categorical_accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()]
    new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return new_model

In [7]:
model = tf.keras.applications.VGG19(include_top=False,input_shape=(112, 112, 3), weights='imagenet')
vgg19 = model_creator(model)
history_vgg19 = vgg19.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)



D:\Anaconda\envs\tf_gpu_final\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
C:\Users\User\AppData\Local\Temp\ipykernel_12048\183852360.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_vgg19 = vgg19.fit_generator(generator=generator_train,


Epoch 1/10
228/228 [==============================] - 133s 550ms/step - loss: 0.3248 - categorical_accuracy: 0.8573 - precision: 0.8573 - recall: 0.8568 - auc: 0.9350 - val_loss: 0.4029 - val_categorical_accuracy: 0.8072 - val_precision: 0.8072 - val_recall: 0.8072 - val_auc: 0.9189
Epoch 2/10
228/228 [==============================] - 123s 539ms/step - loss: 0.1355 - categorical_accuracy: 0.9500 - precision: 0.9500 - recall: 0.9500 - auc: 0.9887 - val_loss: 0.1868 - val_categorical_accuracy: 0.9398 - val_precision: 0.9398 - val_recall: 0.9398 - val_auc: 0.9797
Epoch 3/10
228/228 [==============================] - 126s 549ms/step - loss: 0.0822 - categorical_accuracy: 0.9667 - precision: 0.9669 - recall: 0.9667 - auc: 0.9960 - val_loss: 0.2877 - val_categorical_accuracy: 0.8795 - val_precision: 0.8795 - val_recall: 0.8795 - val_auc: 0.9591
Epoch 4/10
228/228 [==============================] - 127s 556ms/step - loss: 0.0532 - categorical_accuracy: 0.9822 - precision: 0.9822 - recall: 0.

In [8]:
model = tf.keras.applications.InceptionV3(include_top=False,input_shape=(112, 112, 3), weights='imagenet')
inception = model_creator(model)
history_inception = inception.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)



C:\Users\User\AppData\Local\Temp\ipykernel_12048\576356427.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_inception = inception.fit_generator(generator=generator_train,


Epoch 1/10
228/228 [==============================] - 138s 556ms/step - loss: 0.5944 - categorical_accuracy: 0.6816 - precision_1: 0.6815 - recall_1: 0.6813 - auc_1: 0.7533 - val_loss: 0.7951 - val_categorical_accuracy: 0.5904 - val_precision_1: 0.5904 - val_recall_1: 0.5904 - val_auc_1: 0.6564
Epoch 2/10
228/228 [==============================] - 123s 540ms/step - loss: 0.4183 - categorical_accuracy: 0.8019 - precision_1: 0.8018 - recall_1: 0.8016 - auc_1: 0.8893 - val_loss: 0.9222 - val_categorical_accuracy: 0.6988 - val_precision_1: 0.6988 - val_recall_1: 0.6988 - val_auc_1: 0.7329
Epoch 3/10
228/228 [==============================] - 124s 541ms/step - loss: 0.3282 - categorical_accuracy: 0.8554 - precision_1: 0.8554 - recall_1: 0.8552 - auc_1: 0.9353 - val_loss: 1.3350 - val_categorical_accuracy: 0.7470 - val_precision_1: 0.7470 - val_recall_1: 0.7470 - val_auc_1: 0.7805
Epoch 4/10
228/228 [==============================] - 124s 539ms/step - loss: 0.2847 - categorical_accuracy: 0.8

In [9]:
model = tf.keras.applications.MobileNet(include_top=False,input_shape=(112, 112, 3), weights='imagenet')
mobile = model_creator(model)
history_mobile = mobile.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)



C:\Users\User\AppData\Local\Temp\ipykernel_12048\789522733.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_mobile = mobile.fit_generator(generator=generator_train,


Epoch 1/10
228/228 [==============================] - 129s 545ms/step - loss: 0.7890 - categorical_accuracy: 0.6999 - precision_2: 0.6998 - recall_2: 0.6996 - auc_2: 0.7685 - val_loss: 0.9343 - val_categorical_accuracy: 0.6506 - val_precision_2: 0.6506 - val_recall_2: 0.6506 - val_auc_2: 0.7639
Epoch 2/10
228/228 [==============================] - 122s 533ms/step - loss: 0.4853 - categorical_accuracy: 0.8221 - precision_2: 0.8221 - recall_2: 0.8221 - auc_2: 0.8986 - val_loss: 0.7298 - val_categorical_accuracy: 0.7470 - val_precision_2: 0.7470 - val_recall_2: 0.7470 - val_auc_2: 0.8122
Epoch 3/10
228/228 [==============================] - 122s 534ms/step - loss: 0.4050 - categorical_accuracy: 0.8516 - precision_2: 0.8516 - recall_2: 0.8516 - auc_2: 0.9278 - val_loss: 0.7173 - val_categorical_accuracy: 0.7470 - val_precision_2: 0.7470 - val_recall_2: 0.7470 - val_auc_2: 0.8334
Epoch 4/10
228/228 [==============================] - 125s 545ms/step - loss: 0.3462 - categorical_accuracy: 0.8

In [10]:
model = tf.keras.applications.DenseNet121(include_top=False,input_shape=(112, 112, 3), weights='imagenet')
dense = model_creator(model)
history_dense = dense.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)



C:\Users\User\AppData\Local\Temp\ipykernel_12048\3413688818.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_dense = dense.fit_generator(generator=generator_train,


Epoch 1/10
228/228 [==============================] - 150s 583ms/step - loss: 0.5650 - categorical_accuracy: 0.7406 - precision_3: 0.7408 - recall_3: 0.7406 - auc_3: 0.8226 - val_loss: 0.4172 - val_categorical_accuracy: 0.8072 - val_precision_3: 0.8072 - val_recall_3: 0.8072 - val_auc_3: 0.8928
Epoch 2/10
228/228 [==============================] - 129s 562ms/step - loss: 0.3182 - categorical_accuracy: 0.8650 - precision_3: 0.8650 - recall_3: 0.8647 - auc_3: 0.9397 - val_loss: 0.3204 - val_categorical_accuracy: 0.8795 - val_precision_3: 0.8795 - val_recall_3: 0.8795 - val_auc_3: 0.9353
Epoch 3/10
228/228 [==============================] - 129s 565ms/step - loss: 0.2227 - categorical_accuracy: 0.9082 - precision_3: 0.9084 - recall_3: 0.9079 - auc_3: 0.9695 - val_loss: 0.2999 - val_categorical_accuracy: 0.9036 - val_precision_3: 0.9036 - val_recall_3: 0.9036 - val_auc_3: 0.9501
Epoch 4/10
228/228 [==============================] - 129s 564ms/step - loss: 0.1877 - categorical_accuracy: 0.9

In [11]:
model = tf.keras.applications.Xception(include_top=False,input_shape=(112, 112, 3), weights='imagenet')
xcep = model_creator(model)
history_xcep = xcep.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)



C:\Users\User\AppData\Local\Temp\ipykernel_12048\28214634.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_xcep = xcep.fit_generator(generator=generator_train,


Epoch 1/10
228/228 [==============================] - 136s 563ms/step - loss: 0.5511 - categorical_accuracy: 0.7037 - precision_4: 0.7039 - recall_4: 0.7035 - auc_4: 0.7880 - val_loss: 0.4808 - val_categorical_accuracy: 0.7711 - val_precision_4: 0.7683 - val_recall_4: 0.7590 - val_auc_4: 0.8645
Epoch 2/10
228/228 [==============================] - 127s 555ms/step - loss: 0.3380 - categorical_accuracy: 0.8598 - precision_4: 0.8598 - recall_4: 0.8595 - auc_4: 0.9381 - val_loss: 0.3917 - val_categorical_accuracy: 0.8193 - val_precision_4: 0.8193 - val_recall_4: 0.8193 - val_auc_4: 0.9049
Epoch 3/10
228/228 [==============================] - 127s 556ms/step - loss: 0.2360 - categorical_accuracy: 0.9098 - precision_4: 0.9098 - recall_4: 0.9098 - auc_4: 0.9705 - val_loss: 0.4301 - val_categorical_accuracy: 0.7952 - val_precision_4: 0.7952 - val_recall_4: 0.7952 - val_auc_4: 0.8941
Epoch 4/10
228/228 [==============================] - 126s 549ms/step - loss: 0.1831 - categorical_accuracy: 0.9

In [12]:
model = tf.keras.applications.VGG16(include_top=False,input_shape=(112, 112, 3), weights='imagenet')
vgg16 = model_creator(model)
history_vgg16 = vgg16.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)



C:\Users\User\AppData\Local\Temp\ipykernel_12048\388627917.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_vgg16 = vgg16.fit_generator(generator=generator_train,


Epoch 1/10
228/228 [==============================] - 133s 567ms/step - loss: 0.3339 - categorical_accuracy: 0.8502 - precision_5: 0.8502 - recall_5: 0.8500 - auc_5: 0.9319 - val_loss: 0.6138 - val_categorical_accuracy: 0.7470 - val_precision_5: 0.7470 - val_recall_5: 0.7470 - val_auc_5: 0.8384
Epoch 2/10
228/228 [==============================] - 128s 560ms/step - loss: 0.1564 - categorical_accuracy: 0.9363 - precision_5: 0.9366 - recall_5: 0.9363 - auc_5: 0.9851 - val_loss: 0.6489 - val_categorical_accuracy: 0.8072 - val_precision_5: 0.8072 - val_recall_5: 0.8072 - val_auc_5: 0.8929
Epoch 3/10
228/228 [==============================] - 126s 553ms/step - loss: 0.0926 - categorical_accuracy: 0.9680 - precision_5: 0.9680 - recall_5: 0.9680 - auc_5: 0.9947 - val_loss: 1.1376 - val_categorical_accuracy: 0.7831 - val_precision_5: 0.7831 - val_recall_5: 0.7831 - val_auc_5: 0.8368
Epoch 4/10
228/228 [==============================] - 124s 541ms/step - loss: 0.0560 - categorical_accuracy: 0.9